In [1]:
import requests, bs4
import re
import pandas as pd
import time 
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

#### データフレーム作成

In [2]:
cols = ["url","date","venue"]
game_yokohama = pd.DataFrame(index=[], columns=cols)

#### スクレイピング

In [3]:
# index : 1回裏：0120100,0120200,...2回裏：0220100,0220200,...
for i in range(6,12):
    url= 'https://baseball.yahoo.co.jp/npb/teams/3/schedule/?month=2020-0{}'.format(i)
    print(url)
    index = re.search(r'index=\d+', url)
    res = requests.get(url)

    try:
        res.raise_for_status() 
    except requests.exceptions.RequestException:
        print('Not Found Error in {}'.format(index.group())) #Not Foundエラーが出たらその回は終わったとみなす
        #break

    soup = bs4.BeautifulSoup(res.text, "html.parser")
    elems = soup.select('.bb-calendarTable__status')
    elems2 = soup.select('.bb-calendarTable__venue')

    #print(elems)
    for elem,elem2 in zip(elems,elems2):
        url = elem.get('href')
        date = url[38:46]
        venue = elem2.text
        #print(venue)
        game_yokohama.loc[len(game_yokohama)] = [url,date,venue]
    time.sleep(1)

https://baseball.yahoo.co.jp/npb/teams/3/schedule/?month=2020-06
https://baseball.yahoo.co.jp/npb/teams/3/schedule/?month=2020-07
https://baseball.yahoo.co.jp/npb/teams/3/schedule/?month=2020-08
https://baseball.yahoo.co.jp/npb/teams/3/schedule/?month=2020-09
https://baseball.yahoo.co.jp/npb/teams/3/schedule/?month=2020-010
https://baseball.yahoo.co.jp/npb/teams/3/schedule/?month=2020-011


In [4]:
# 中止も含むため少し多い
game_yokohama

,url,date,venue
0,https://baseball.yahoo.co.jp/npb/game/20200619...,20200619,横浜
1,https://baseball.yahoo.co.jp/npb/game/20200620...,20200620,横浜
2,https://baseball.yahoo.co.jp/npb/game/20200621...,20200621,横浜
3,https://baseball.yahoo.co.jp/npb/game/20200623...,20200623,横浜
4,https://baseball.yahoo.co.jp/npb/game/20200624...,20200624,横浜
...,...,...,...
119,https://baseball.yahoo.co.jp/npb/game/20201103...,20201103,ナゴヤドーム
120,https://baseball.yahoo.co.jp/npb/game/20201104...,20201104,ナゴヤドーム
121,https://baseball.yahoo.co.jp/npb/game/20201105...,20201105,ナゴヤドーム
122,https://baseball.yahoo.co.jp/npb/game/20201111...,20201111,甲子園


In [5]:
game_yokohama.to_csv("game_yokohama.csv")

In [ ]:
#https://baseball.yahoo.co.jp/npb/game/2020062802/index の index→scoreで一球速報

In [18]:
cols = ["index","date","top","left","pitcher","batter","batter_rl","type","speed","result"]
ball_distribution_yokohama_attack = pd.DataFrame(index=[], columns=cols) #横浜の攻撃
ball_distribution_yokohama_defence = pd.DataFrame(index=[], columns=cols) #横浜の守備

In [19]:
for base_url,date,venue in  zip(list(game_yokohama["url"]),list(game_yokohama["date"]),list(game_yokohama["venue"])):
    base_url = base_url[:-5]+'score'
    # index : 1回裏：0120100,0120200,...2回裏：0220100,0220200,...
    for inning in range(1,11): #1-10回
        print(inning)
        for b_count in range(100,10000,100): #打席
            url= '{}?index=0{}10{}'.format(base_url,inning,b_count)
            #print(url)
            index = re.search(r'index=\d+', url)
            res = requests.get(url)

            try:
                res.raise_for_status() 
            except requests.exceptions.RequestException:
                print('Not Found Error in {}'.format(index.group())) #Not Foundエラーが出たらその回は終わったとみなす
                break

            soup = bs4.BeautifulSoup(res.text, "html.parser")
            elems = soup.select('.bb-icon__ballCircle')
            elems1 = soup.select('#gm_rslt .bb-splitsTable__data a') #投手、打者
            elems2 = soup.select('.bb-splitsTable__data--ballType') #球種
            elems3 = soup.select('.bb-splitsTable__data--speed') #球速
            elems4 = soup.select('.bb-splitsTable__data--result') #結果
            elems5 = soup.select('#gm_rslt .bb-splitsTable__data') #打者左右
            #print(elems)
            for elem,elem2,elem3,elem4 in zip(elems,elems2,elems3,elems4):
                s = str(elem)
                ball_type = elem2.string
                ball_speed = elem3.string
                top = re.search(r'top:\d+px', s)
                left = re.search(r'left:\d+px', s)
                batter = elems1[0].string
                pitcher = elems1[1].string
                result = elem4.text
                rl = elems5[1].string
                #print(s1)  #投手、打者
                if top != None and left != None:    
                    if venue != "横浜": #ホーム以外の表(横浜の攻撃)
                        ball_distribution_yokohama_attack.loc[len(ball_distribution_yokohama_attack)] = [int(index.group()[6:]),date,int(top.group()[4:][:-2]),int(left.group()[5:][:-2]),batter,pitcher,rl,ball_type,ball_speed[:-4],result]
                    else: #ホームの表(横浜の守備)
                        ball_distribution_yokohama_defence.loc[len(ball_distribution_yokohama_defence)] = [int(index.group()[6:]),date,int(top.group()[4:][:-2]),int(left.group()[5:][:-2]),batter,pitcher,rl,ball_type,ball_speed[:-4],result]
            time.sleep(1)
            
        for b_count in range(100,10000,100): #打席
            url= '{}?index=0{}20{}'.format(base_url,inning,b_count)
            #print(url)
            index = re.search(r'index=\d+', url)
            res = requests.get(url)

            try:
                res.raise_for_status() 
            except requests.exceptions.RequestException:
                print('Not Found Error in {}'.format(index.group())) #Not Foundエラーが出たらその回は終わったとみなす
                break

            soup = bs4.BeautifulSoup(res.text, "html.parser")
            elems = soup.select('.bb-icon__ballCircle')
            elems1 = soup.select('#gm_rslt .bb-splitsTable__data a') #投手、打者
            elems2 = soup.select('.bb-splitsTable__data--ballType') #球種
            elems3 = soup.select('.bb-splitsTable__data--speed') #球速
            elems4 = soup.select('.bb-splitsTable__data--result') #結果
            elems5 = soup.select('#gm_rslt .bb-splitsTable__data') #打者左右
            #print(elems)
            for elem,elem2,elem3,elem4 in zip(elems,elems2,elems3,elems4):
                s = str(elem)
                ball_type = elem2.string
                ball_speed = elem3.string
                top = re.search(r'top:\d+px', s)
                left = re.search(r'left:\d+px', s)
                batter = elems1[0].string
                pitcher = elems1[1].string
                result = elem4.text
                rl = elems5[1].string
                #print(s1)  #投手、打者
                if top != None and left != None:
                    if venue != "横浜": #ホーム以外の裏(横浜の守備)
                        ball_distribution_yokohama_defence.loc[len(ball_distribution_yokohama_defence)] = [int(index.group()[6:]),date,int(top.group()[4:][:-2]),int(left.group()[5:][:-2]),pitcher,batter,rl,ball_type,ball_speed[:-4],result]
                    else: #ホームの裏(横浜の攻撃)
                        ball_distribution_yokohama_attack.loc[len(ball_distribution_yokohama_attack)] = [int(index.group()[6:]),date,int(top.group()[4:][:-2]),int(left.group()[5:][:-2]),pitcher,batter,rl,ball_type,ball_speed[:-4],result]
            time.sleep(1)

1
Not Found Error in index=0110500
Not Found Error in index=0120400
2
Not Found Error in index=0210500
Not Found Error in index=0220500
3
Not Found Error in index=0310400
Not Found Error in index=0320400
4
Not Found Error in index=0410500
Not Found Error in index=0420400
5
Not Found Error in index=0510800
Not Found Error in index=0520400
6
Not Found Error in index=0610400
Not Found Error in index=0620400
7
Not Found Error in index=0710500
Not Found Error in index=0720500
8
Not Found Error in index=0810400
Not Found Error in index=0820500
9
Not Found Error in index=0910700
Not Found Error in index=0920600
10
Not Found Error in index=01010100
Not Found Error in index=01020100
1
Not Found Error in index=0110600
Not Found Error in index=0120600
2
Not Found Error in index=0210600
Not Found Error in index=0220400
3
Not Found Error in index=0310400
Not Found Error in index=0320400
4
Not Found Error in index=0410600
Not Found Error in index=0420700
5
Not Found Error in index=0510400
Not Found 

Not Found Error in index=0110600
Not Found Error in index=0120800
2
Not Found Error in index=0210400
Not Found Error in index=0220400
3
Not Found Error in index=03101000
Not Found Error in index=0320900
4
Not Found Error in index=0410600
Not Found Error in index=0420400
5
Not Found Error in index=0510600
Not Found Error in index=0520400
6
Not Found Error in index=0610500
Not Found Error in index=0620500
7
Not Found Error in index=0710400
Not Found Error in index=0720500
8
Not Found Error in index=0810500
Not Found Error in index=0820500
9
Not Found Error in index=0910400
Not Found Error in index=0920500
10
Not Found Error in index=01010100
Not Found Error in index=01020100
1
Not Found Error in index=0110500
Not Found Error in index=0120400
2
Not Found Error in index=0210500
Not Found Error in index=0220600
3
Not Found Error in index=0310600
Not Found Error in index=0320600
4
Not Found Error in index=0410500
Not Found Error in index=0420400
5
Not Found Error in index=0510700
Not Found E

Not Found Error in index=0110500
Not Found Error in index=0120400
2
Not Found Error in index=0210600
Not Found Error in index=0220600
3
Not Found Error in index=0310500
Not Found Error in index=0320500
4
Not Found Error in index=0410400
Not Found Error in index=0420400
5
Not Found Error in index=0510400
Not Found Error in index=0520600
6
Not Found Error in index=0610400
Not Found Error in index=0620400
7
Not Found Error in index=0710100
Not Found Error in index=0720100
8
Not Found Error in index=0810100
Not Found Error in index=0820100
9
Not Found Error in index=0910100
Not Found Error in index=0920100
10
Not Found Error in index=01010100
Not Found Error in index=01020100
1
Not Found Error in index=0110500
Not Found Error in index=0120400
2
Not Found Error in index=0210800
Not Found Error in index=0220400
3
Not Found Error in index=0310400
Not Found Error in index=0320400
4
Not Found Error in index=0410500
Not Found Error in index=0420500
5
Not Found Error in index=0510500
Not Found Er

Not Found Error in index=0110500
Not Found Error in index=0120500
2
Not Found Error in index=0210500
Not Found Error in index=0220400
3
Not Found Error in index=0310500
Not Found Error in index=0320400
4
Not Found Error in index=0410400
Not Found Error in index=0420800
5
Not Found Error in index=0510500
Not Found Error in index=0520400
6
Not Found Error in index=0610500
Not Found Error in index=0620600
7
Not Found Error in index=0710800
Not Found Error in index=0720500
8
Not Found Error in index=0810600
Not Found Error in index=0820600
9
Not Found Error in index=0910600
Not Found Error in index=0920400
10
Not Found Error in index=01010100
Not Found Error in index=01020100
1
Not Found Error in index=0110600
Not Found Error in index=0120400
2
Not Found Error in index=0210600
Not Found Error in index=0220400
3
Not Found Error in index=0310500
Not Found Error in index=0320500
4
Not Found Error in index=0410500
Not Found Error in index=0420800
5
Not Found Error in index=0510600
Not Found Er

Not Found Error in index=0110400
Not Found Error in index=0120700
2
Not Found Error in index=0210600
Not Found Error in index=0220600
3
Not Found Error in index=0310400
Not Found Error in index=0320400
4
Not Found Error in index=0410500
Not Found Error in index=0420500
5
Not Found Error in index=0510400
Not Found Error in index=0520600
6
Not Found Error in index=0610600
Not Found Error in index=0620400
7
Not Found Error in index=0710400
Not Found Error in index=0720900
8
Not Found Error in index=0810600
Not Found Error in index=0820500
9
Not Found Error in index=0910500
Not Found Error in index=0920100
10
Not Found Error in index=01010100
Not Found Error in index=01020100
1
Not Found Error in index=0110500
Not Found Error in index=0120500
2
Not Found Error in index=0210400
Not Found Error in index=0220500
3
Not Found Error in index=0310800
Not Found Error in index=0320400
4
Not Found Error in index=0410900
Not Found Error in index=0420400
5
Not Found Error in index=0510400
Not Found Er

...

Not Found Error in index=0110400
Not Found Error in index=0120500
2
Not Found Error in index=0210600
Not Found Error in index=0220500
3
Not Found Error in index=0310700
Not Found Error in index=0320800
4
Not Found Error in index=0410700
Not Found Error in index=0420700
5
Not Found Error in index=0510600
Not Found Error in index=0520500
6
Not Found Error in index=0610400
Not Found Error in index=0620800
7
Not Found Error in index=0710600
Not Found Error in index=0720400
8
Not Found Error in index=0810500
Not Found Error in index=0820500
9
Not Found Error in index=0910600
Not Found Error in index=0920400
10
Not Found Error in index=01010100
Not Found Error in index=01020100
1
Not Found Error in index=0110500
Not Found Error in index=0120400
2
Not Found Error in index=0210600
Not Found Error in index=0220400
3
Not Found Error in index=0310400
Not Found Error in index=0320400
4
Not Found Error in index=0410900
Not Found Error in index=0420400
5
Not Found Error in index=0510500
Not Found Er

Not Found Error in index=0110400
Not Found Error in index=0120400
2
Not Found Error in index=0210600
Not Found Error in index=0220400
3
Not Found Error in index=0310500
Not Found Error in index=0320500
4
Not Found Error in index=0410400
Not Found Error in index=0420400
5
Not Found Error in index=0510600
Not Found Error in index=0520500
6
Not Found Error in index=0610700
Not Found Error in index=0620400
7
Not Found Error in index=0710800
Not Found Error in index=0720500
8
Not Found Error in index=0810600
Not Found Error in index=0820500
9
Not Found Error in index=0910500
Not Found Error in index=0920600
10
Not Found Error in index=01010100
Not Found Error in index=01020100
1
Not Found Error in index=0110400
Not Found Error in index=0120400
2
Not Found Error in index=0210600
Not Found Error in index=0220600
3
Not Found Error in index=0310400
Not Found Error in index=0320600
4
Not Found Error in index=0410700
Not Found Error in index=0420600
5
Not Found Error in index=0510500
Not Found Er

Not Found Error in index=0110700
Not Found Error in index=0120700
2
Not Found Error in index=0210500
Not Found Error in index=0220400
3
Not Found Error in index=0310400
Not Found Error in index=0320400
4
Not Found Error in index=0410400
Not Found Error in index=0420500
5
Not Found Error in index=0510400
Not Found Error in index=0520600
6
Not Found Error in index=0610900
Not Found Error in index=0620600
7
Not Found Error in index=0710500
Not Found Error in index=0720500
8
Not Found Error in index=0810400
Not Found Error in index=0820400
9
Not Found Error in index=0910600
Not Found Error in index=0920400
10
Not Found Error in index=01010100
Not Found Error in index=01020100
1
Not Found Error in index=0110400
Not Found Error in index=0120700
2
Not Found Error in index=0210600
Not Found Error in index=0220700
3
Not Found Error in index=0310400
Not Found Error in index=0320700
4
Not Found Error in index=0410400
Not Found Error in index=0420800
5
Not Found Error in index=0510400
Not Found Er

Not Found Error in index=0110600
Not Found Error in index=0120500
2
Not Found Error in index=0210500
Not Found Error in index=0220600
3
Not Found Error in index=0310700
Not Found Error in index=0320500
4
Not Found Error in index=0410400
Not Found Error in index=0420400
5
Not Found Error in index=0510500
Not Found Error in index=0520600
6
Not Found Error in index=0610400
Not Found Error in index=0620600
7
Not Found Error in index=0710700
Not Found Error in index=0720700
8
Not Found Error in index=0810500
Not Found Error in index=0820600
9
Not Found Error in index=0910600
Not Found Error in index=0920100
10
Not Found Error in index=01010100
Not Found Error in index=01020100
1
Not Found Error in index=0110400
Not Found Error in index=0120600
2
Not Found Error in index=0210400
Not Found Error in index=0220400
3
Not Found Error in index=0310400
Not Found Error in index=0320500
4
Not Found Error in index=0410400
Not Found Error in index=0420400
5
Not Found Error in index=0510400
Not Found Er

Not Found Error in index=0110400
Not Found Error in index=0120500
2
Not Found Error in index=0210400
Not Found Error in index=0220400
3
Not Found Error in index=0310500
Not Found Error in index=0320500
4
Not Found Error in index=0410400
Not Found Error in index=0420900
5
Not Found Error in index=0510500
Not Found Error in index=0520500
6
Not Found Error in index=0610900
Not Found Error in index=0620500
7
Not Found Error in index=0710400
Not Found Error in index=0720500
8
Not Found Error in index=0810400
Not Found Error in index=0820500
9
Not Found Error in index=0910400
Not Found Error in index=0920100
10
Not Found Error in index=01010100
Not Found Error in index=01020100
1
Not Found Error in index=0110500
Not Found Error in index=0120400
2
Not Found Error in index=0210600
Not Found Error in index=0220700
3
Not Found Error in index=0310500
Not Found Error in index=0320500
4
Not Found Error in index=0410500
Not Found Error in index=0420600
5
Not Found Error in index=0510500
Not Found Er

In [25]:
ball_distribution_yokohama_attack

,index,date,top,left,pitcher,batter,batter_rl,type,speed,result
0,120100,20200619,155,60,大瀬良 大地,梶谷 隆幸,左,ストレート,138,\n 右飛\n ...
1,120200,20200619,158,70,大瀬良 大地,乙坂 智,左,ストレート,145,\n 二ゴロ\n ...
2,120300,20200619,92,6,大瀬良 大地,ソト,右,ストレート,145,\n ボール\n ...
3,120300,20200619,180,28,大瀬良 大地,ソト,右,カーブ,116,\n ボール\n ...
4,120300,20200619,12,88,大瀬良 大地,ソト,右,カットボール,139,\n ボール\n ...
...,...,...,...,...,...,...,...,...,...,...
16873,920500,20201114,167,124,田口 麗斗,宮本 秀明,左,スライダー,131,\n 空振り\n ...
16874,920500,20201114,18,97,田口 麗斗,宮本 秀明,左,スライダー,132,\n ボール\n ...
16875,920500,20201114,46,29,田口 麗斗,宮本 秀明,左,スライダー,132,\n 見逃し\n ...
16876,920500,20201114,155,100,田口 麗斗,宮本 秀明,左,スライダー,133,\n 投安\n ...


In [26]:
ball_distribution_yokohama_defence

,index,date,top,left,pitcher,batter,batter_rl,type,speed,result
0,110100,20200619,156,92,今永 昇太,ピレラ,左,ストレート,149,\n ボール\n ...
1,110100,20200619,158,69,今永 昇太,ピレラ,左,ストレート,149,\n 空振り\n ...
2,110100,20200619,20,9,今永 昇太,ピレラ,左,チェンジアップ,137,\n ボール\n ...
3,110100,20200619,90,126,今永 昇太,ピレラ,左,カットボール,138,\n ファウル\n ...
4,110100,20200619,44,26,今永 昇太,ピレラ,左,チェンジアップ,136,\n 右安\n ...
...,...,...,...,...,...,...,...,...,...,...
16972,910300,20201114,75,37,三嶋 一輝,桜井 俊貴,右,ストレート,147,\n 捕ゴロ\n ...
16973,910400,20201114,18,57,三嶋 一輝,湯浅 大,右,カーブ,126,\n ボール\n ...
16974,910400,20201114,72,58,三嶋 一輝,湯浅 大,右,スライダー,141,\n 空振り\n ...
16975,910400,20201114,51,32,三嶋 一輝,湯浅 大,右,ストレート,151,\n 空振り\n ...


In [22]:
ball_distribution_yokohama_attack.to_csv('ball_distribution_yokohama_attack_cp932.csv',index=False,encoding='cp932')
ball_distribution_yokohama_defence.to_csv('ball_distribution_yokohama_defence_cp932.csv',index=False,encoding='cp932')

In [23]:
ball_distribution_yokohama_attack.to_csv('ball_distribution_yokohama_attack.csv',index=False)
ball_distribution_yokohama_defence.to_csv('ball_distribution_yokohama_defence.csv',index=False)